From https://github.com/blackmad/neighborhoods/blob/master/paris.geojson we export the geojson code for Paris districts.

Note that the initial raw content would only provide one document when uploaded as a collection in MongoDB Compass. Hence, we followed the second procedure established in troubleshooting section of lectures (https://github.com/Ironhack-data-bcn-january-2023/lectures/blob/main/troubleshooting/importing-into-db/importing-geojsons-into-mongodb.md), and manually removed the first curly brackets pair of the code, in order to maintain only the list with all polygons instead of a multipolygonal dictionary. After that, we successfully uploaded the JSON file to Paris collection in MongoDB.

# With geojson not including features (imported from Mongo DB)

In [442]:
# Libraries
from pymongo import MongoClient
import os
import requests
import json
from dotenv import load_dotenv
from bson.json_util import dumps
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from bs4 import BeautifulSoup
import shapely
from shapely import Polygon
from shapely.geometry import shape, Point

from paris_geoqueries import *

In [48]:
load_dotenv()

True

In [49]:
# Foursquare key to get API queries.
foursquare_key = os.getenv('fsq_key')

In [2]:
# Paris collection
client = MongoClient('localhost:27017')
db = client.get_database('ironhack')
paris = db.get_collection('paris')
paris = list(paris.find())

In [445]:
# Dataframe with each district area in squared kilometers.
area_info = arrondissement_scraping()
area_info

/var/folders/lr/stnkft6j5ps6pysrw7v0cgp80000gn/T/ipykernel_48634/1719465365.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  districts_area.rename(columns={'Area (km  2  )': 'Area'}, inplace = True, errors = 'raise')
/var/folders/lr/stnkft6j5ps6pysrw7v0cgp80000gn/T/ipykernel_48634/1719465365.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  districts_area.sort_values(by = ['Name'], ascending = True, inplace = True)


,Name,Area
0,Batignolles-Monceau,5.6690
1,Bourse,1.3975
2,Butte-Montmartre,6.0050
3,Buttes-Chaumont,6.7860
4,Entrepôt,2.8920
5,Gobelins,7.1460
6,Hôtel-de-Ville,1.3975
7,Louvre,1.3975
8,Luxembourg,2.1540
9,Ménilmontant,5.9840


# ON THE MAP, AFTER MAKING SOME DATAFRAME WITH VARIABLES AND FREQUENCIES AND DEFINING THE PONDERATION, PLOT COLORED DISTRICTS WITH SOME SCALE!

In [446]:
# Feature geojson is the one used to plot
with open('feature.geojson') as geo_file:
    geo_feature = json.load(geo_file)
    
paris_map = Map(location = [48.86, 2.35], zoom_start = 11)
folium.Choropleth(
    geo_data=geo_feature,
    key_on="feature.properties.name",
).add_to(paris_map)
paris_map

# FOURSQUARE API REQUESTS!!! 
WILL PROVIDE SOME COORDINATES THAT WILL HAVE TO BE PASSED TO DISTRICT CHECKER!

In [444]:
vegan_restaurants = foursquare_query('Vegan restaurants', 'Paris', 15)
vegan_restaurants

,Coordinates,Name,Type
0,POINT (2.367256 48.852804),Strada Café,Café
1,POINT (2.370188 48.868127),Strada Café,Café
2,POINT (2.369075 48.870262),Strada Café,Café
3,POINT (2.373175 48.861708),Strada Café,Café
4,POINT (2.358367 48.861547),Strada Café,Café
5,POINT (2.344854 48.878986),Strada Café,Café
6,POINT (2.317107 48.855784),Strada Café,Café
7,POINT (2.293396 48.872043),Strada Café,Café
8,POINT (2.363137 48.873239),Strada Café,Café
9,POINT (2.345682 48.860489),Strada Café,Café


# IMPORTANT PIECE OF CODE: CHECK IF A POINT BELONGS TO A DISTRICT!!!

## Density criteria
### Spot finder gets a df with the count of queried instances. This df is plotted into a map
The objective is to set a punctuation according to instances and district area -> density of instances!

In [441]:
vvvv = spot_finder(vegan_restaurants)
vvvv

,District,Density
0,Batignolles-Monceau,0.000000
1,Bourse,0.000000
2,Butte-Montmartre,0.000000
3,Buttes-Chaumont,0.000000
4,Enclos-St-Laurent,1.383126
5,Gobelins,0.000000
6,Hôtel-de-Ville,0.715564
7,Louvre,1.431127
8,Luxembourg,0.000000
9,Ménilmontant,0.000000


In [436]:
district_distribution(vegan_distribution)

# Distance criteria
Airports, dog hairdresser and basketball stadiums

In [334]:
dist_from_center_airport = distance_criteria('Airport')

In [335]:
dist_from_center_airport

,Points,District,Distance
0,0,Reuilly,2421
1,1,Butte-Montmartre,1398
2,2,Buttes-Chaumont,1291
3,3,Panthéon,1110
4,4,Luxembourg,1062
5,5,Popincourt,1056
6,6,Vaugirard,851
7,7,Opéra,842
8,8,Batignolles-Monceau,833
9,9,Palais-Bourbon,714
